In [8]:
import torch
import tokenization
import parse_pdb
import pandas as pd


In [12]:
data = parse_pdb.load_pdb(['protein'], data_loc='./data/pdb_seqres.txt').sort_values(by='Name')
data

,Name,Sequence,MolType,length
60860,,GPLGSMALRACGLIIFRRCLIPKVDNNAIEFLLLQASDGIHHWTPP...,protein,153.0
60861,,GPLGSMALRACGLIIFRRCLIPKVDNNAIEFLLLQASDGIHHWTPP...,protein,153.0
41931,,MLVVPAIDLFRGKVARMIKGRKENTIFYEKDPVELVEKLIEEGFTL...,protein,241.0
60862,,GPLGSMALRACGLIIFRRCLIPKVDNNAIEFLLLQASDGIHHWTPP...,protein,153.0
649221,,MSFRFGQCLIKPSVVFLKTELSFALVNRKPVVPGHVLVCPLRPVER...,protein,163.0
...,...,...,...,...
128089,zinc peptidase active subunit,ADPAASTFETTLPNGLKVVVREDHRAPTLVHMVWYRVGSMDETTGT...,protein,437.0
128087,zinc peptidase inactive subunit,AIKIEHWTAPSGAQVYYVENRTLPMLDVQVDFDAGSAREPADQVGV...,protein,424.0
128086,zinc peptidase inactive subunit,AIKIEHWTAPSGAQVYYVENRTLPMLDVQVDFDAGSAREPADQVGV...,protein,424.0
239740,zinc-dependent metalloproteinase,QQRFPQRYVMLAIVADHGMVTKYSGNSSAITTRVHQMVSHVTEMYS...,protein,202.0


In [6]:
data.Name.unique().shape

(90313,)